In [ ]:
%load_ext watermark
%watermark -a 'Christian Schuhegger' -u -d -v -p numpy,xarray,scipy,pandas,matplotlib,seaborn,qgrid,rpy2,libpgm,pgmpy,networkx,graphviz,pybnl,pytest

In [ ]:
%matplotlib inline
import numpy as np, pandas as pd, xarray as xr, matplotlib.pyplot as plt, seaborn as sns
import networkx as nx, graphviz, networkx.algorithms.dag

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

sns.set()

In [ ]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport pybnl.bn

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'C')

import rpy2, rpy2.rinterface, rpy2.robjects, rpy2.robjects.packages, rpy2.robjects.lib, rpy2.robjects.lib.grid, \
    rpy2.robjects.lib.ggplot2, rpy2.robjects.pandas2ri, rpy2.interactive.process_revents, \
    rpy2.interactive, rpy2.robjects.lib.grdevices
# rpy2.interactive.process_revents.start()
rpy2.robjects.pandas2ri.activate()

In [ ]:
%%R -o marks
library(bnlearn)
data(marks)

In [ ]:
marks.head()

* [networkx](https://github.com/networkx/networkx)
* [tutorial](https://networkx.github.io/documentation/stable/tutorial.html)
* [graphviz](https://github.com/xflr6/graphviz)
* [graphviz.readthedocs](https://graphviz.readthedocs.io/en/stable/)
* [examples/notebook.ipynb](https://github.com/xflr6/graphviz/blob/master/examples/notebook.ipynb)

In [ ]:
dg = nx.DiGraph()
# G.add_node(1)
dg.add_nodes_from(list(marks.columns))
dg.add_edges_from([
    ['STAT', 'ANL'],
    ['STAT', 'ALG'],
    ['ANL', 'ALG'],
    ['ALG', 'MECH'],
    ['ALG', 'VECT'],
    ['VECT', 'MECH'],
])

In [ ]:
list(nx.connected_components(dg.to_undirected()))

In [ ]:
nx.draw(dg, with_labels=True)

In [ ]:
dg_dot = graphviz.Digraph(comment='disease-network')
for node in dg.nodes():
    dg_dot.node(node)

for edge in dg.edges():
    dg_dot.edge(edge[0],edge[1])

dg_dot

In [ ]:
list(networkx.algorithms.dag.topological_sort(dg))

In [ ]:
list(dg.nodes()),list(dg.edges())

In [ ]:
nds = rpy2.robjects.StrVector(list(dg.nodes()))

In [ ]:
connectivity_matrix = xr.DataArray(np.zeros((len(marks.columns), len(marks.columns))), 
             dims = ['from','to'], 
             coords= {'from': list(marks.columns), 'to': list(marks.columns)})
for edge in dg.edges():
    connectivity_matrix.loc[{'from': edge[0], 'to': edge[1]}] = 1
np_connectivity_matrix = np.array(connectivity_matrix.values)
np_connectivity_matrix

In [ ]:
%%R -i np_connectivity_matrix -i nds
dag = empty.graph(nds)
amat(dag) = np_connectivity_matrix
dag

In [ ]:
%%R
moral(dag)

In [ ]:
%%R
node.ordering(dag)

In [ ]:
%%R
nbr(dag, "ANL")

In [ ]:
%%R
mb(dag, "ANL")

In [ ]:
%%R
score(dag,data=marks,type="loglik-g")

In [ ]:
%%R
vstructs(dag)

In [ ]:
%%R
vstructs(dag, moral=TRUE)

In [ ]:
%%R
cpdag(dag)

[Principled Computational Methods for the Validation and Discovery of Genetic Regulatory Networks](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.4.6170) by Alexander John Hartemink, 2001

In [ ]:
%%R -o dmarks1 -o dmarks2
dmarks1 = discretize(marks, breaks=3, method="interval")
dmarks2 = discretize(marks, breaks=3, method="hartemink", ibreaks=5, idisc="quantile")

In [ ]:
%%R
levels(dmarks1$MECH)

In [ ]:
%%R
levels(dmarks2$MECH)

In [ ]:
display_side_by_side(dmarks1.head(), dmarks2.head())

In [ ]:
fig=plt.figure(figsize=(15, 8), dpi= 80, facecolor='w', edgecolor='k')
ax  = plt.subplot(1,1,1)
dmarks1['MECH'].groupby(dmarks1['MECH']).agg(['count']).plot.bar(ax=ax);

In [ ]:
import dsbasics.bin

In [ ]:
x = marks['MECH'].values
X = x.reshape(-1,1)

bbt1 = dsbasics.bin.BayesianBlocksBinTransformer(p0=0.05)
bbt1.fit(X)

bbt2 = dsbasics.bin.DecisionTreeBinTransformer(max_leaf_nodes=3)
bbt2.fit(X)


fig=plt.figure(figsize=(15, 8), dpi= 80, facecolor='w', edgecolor='k')
ax  = plt.subplot(1,1,1)

ax.hist(x,bins=40, histtype='stepfilled', alpha=0.2, density=True)
ax.hist(x, bins=bbt1.bins_[0], histtype='step', color='black', density=True)
ax.hist(x, bins=bbt2.bins_[0], histtype='step', color='green', density=True)
ax.set_ylabel('P(MECH)')
# ax.set_xlim(0.0,400.0)
ax.set_xlabel('MECH')

In [ ]:
x = dmarks1['MECH']
c = pd.Categorical(x)
c.categories

In [ ]:
%%R
write.csv(marks, file = "marks.csv")
head(marks)

In [ ]:
pd_marks = pd.read_csv('marks.csv', index_col=0).astype(np.float64)
pd_marks.head()

In [ ]:
# %%R -i pd_marks
# dmarks2_ = discretize(pd_marks, breaks=3, method="hartemink", ibreaks=5, idisc="quantile")
# head(dmarks2_)

In [ ]:
df = pybnl.bn.discretize(pd_marks)
df.head()

In [ ]:
df.dtypes

In [ ]:
df['MECH'].cat.categories

In [ ]:
df['MECH'].groupby(df['MECH']).agg(['count']).plot.bar()

In [ ]:
df['MECH'].head()

In [ ]:
bn = pybnl.bn.NetAndDataDiscreteBayesNetwork(dg, df)

In [ ]:
bn.model_string

In [ ]:
print(bn.rnet)

In [ ]:
print(bn.rfit)

In [ ]:
print(bn.grain)

In [ ]:
df['STAT'].cat.categories[0]

In [ ]:
evidence = dict(ALG=df['ALG'].cat.categories[0])
nodes_to_query = ['STAT']
bn.exact_query(evidence, nodes_to_query, only_python_result=True)

In [ ]:
bn.write_net('marks.net')

In [ ]:
# TODO
# Convert network from R to python xarray and pandas and allow for save and load
# Implement structure learning
# Ask question about discretization score available in bnlearn discretize?
# Plot discretization levels
# Implement error checking
#  Ensure that queries only use valid variables and values for these variables
# Implement unit tests
# Implement approximate queries
# Implement model averaging, cross validation, ...
# Implement ames house price example
#  Arc strengths
# Implement own problem with LLT dependencies
# Plot graphs as equivalence structures (cpdag) and use cextend for consistent extension
# Implement sampling from a network

In [ ]:
rfit = bn.rfit

In [ ]:
%%R -i rfit
# nodes(rfit)
# colnames, rownames, dimnames
# dimnames(rfit$STAT$prob)[1]
# dimnames(rfit$ANL$prob)[1]
dimnames(rfit$ANL$prob)

In [ ]:
%%R -o anl_prob
anl_prob = rfit$ANL$prob

In [ ]:
rfit.rx('ANL')

In [ ]:
rpy2.robjects.pandas2ri.ri2py(anl_prob)

In [ ]:
anl_prob

In [ ]:
anl_prob.names.rx('ANL').names[0]

In [ ]:
anl_prob.names

In [ ]:
anl_prob.names

In [ ]:
rfit.rx('ANL')[0].rx('prob')[0].names.rx('ANL')[0]

In [ ]:
rpy2.robjects.pandas2ri.ri2py(rfit.rx('ANL')[0].rx('prob')[0])

In [ ]:
rfit.rx('ANL')[0].rx('prob')[0].names.rx('ANL')[0]

In [ ]:
rfit.rx('ANL')[0].rx('prob')[0].names

In [ ]:
rfit.rx('STAT')[0].rx('prob')[0].names[0][0]

In [ ]:
ds,lpd = pybnl.bn.convert_to_xarray_dataset(bn.rfit)
ds

In [ ]:
lpd['cptALG']

In [ ]:
ds['cptSTAT']

In [ ]:
ds['cptANL']

In [ ]:
ds['cptANL'].loc[{'STAT': '[9,29.4]'}]

In [ ]:
ds['cptALG']

In [ ]:
ds['cptALG'].loc[{'ANL': '(9,47]', 'STAT': '[9,29.4]'}]

In [ ]:
ds['cptALG'].loc[{'ANL': '(47,60.2]', 'STAT': '(29.4,44]'}]

In [ ]:
ds['cptALG'].values[:,1,1]

In [ ]:
ds['cptALG'].values

In [ ]:
ds['cptALG'].loc[{'ANL': '(60.2,70]', 'STAT': '[9,29.4]'}]

In [ ]:
ds['cptALG'].transpose('ANL', 'STAT', 'ALG').stack(idx=['ANL', 'STAT', 'ALG']).to_pandas()#.reset_index()

In [ ]:
ds.to_netcdf('marks.nc')

* [NetCDF](https://www.unidata.ucar.edu/software/netcdf/software.html) viewers
* use `ncdump` from the `netcdf-bin` package

In [ ]:
tmp = xr.open_dataset('marks.nc', autoclose=True)
tmp

In [ ]:
tmp.equals(ds)

In [ ]:
def tmpfn():
    for ar in tmp.data_vars:
        return ar
tmpfn()

In [ ]:
bn.write_netcdf('marks.nc')

In [ ]:
bn1 = pybnl.bn.bnnet_from_netcdf_file('marks.nc')

In [ ]:
bn1.to_xrds().equals(bn.to_xrds())

In [ ]:
bn.to_xrds()

In [ ]:
tmp = xr.open_dataset('marks.nc',autoclose=True)
tmp.equals(bn.to_xrds())

In [ ]:
tmp

In [ ]:
bn1.to_xrds().equals(tmp)

In [ ]:
bn1.to_dtcpm_dict()['cptALG'].sort_values(['ANL', 'STAT', 'ALG'])

In [ ]:
p1 = bn1.to_dtcpm_dict()['cptALG'].sort_values(['ANL', 'STAT', 'ALG'])['p']
p1.head()

In [ ]:
bn.to_dtcpm_dict()['cptALG'].sort_values(['ANL', 'STAT', 'ALG'])

In [ ]:
p = bn.to_dtcpm_dict()['cptALG'].sort_values(['ANL', 'STAT', 'ALG'])['p']
p.head()

In [ ]:
p.values == p1.values

In [ ]:
bn.to_xrds()['cptALG']

In [ ]:
np.all(bn1.to_xrds()['cptALG'] == bn.to_xrds()['cptALG'])

In [ ]:
tmp = bn.rfit

In [ ]:
%%R -i tmp
arcs(tmp)

In [ ]:
%%R
arcs(cpdag(tmp))

In [ ]:
%%R
amat(tmp)

In [ ]:
%%R
amat(cpdag(tmp))

In [ ]:
dg = nx.DiGraph()
# G.add_node(1)
dg.add_nodes_from(list(marks.columns))
dg.add_edges_from([
    ['STAT', 'ANL'],
    ['STAT', 'ALG'],
    ['ANL', 'ALG'],
    ['ALG', 'MECH'],
    ['ALG', 'VECT'],
    ['VECT', 'MECH'],
])
dg

In [ ]:
dg_dot = graphviz.Digraph(comment='disease-network')
for node in dg.nodes():
    dg_dot.node(node)

for edge in dg.edges():
    dg_dot.edge(edge[0],edge[1])

print(dg_dot.source)

In [ ]:
dg_dot = graphviz.Digraph(comment='disease-network')
for node in dg.nodes():
    dg_dot.node(node)

for edge in dg.edges():
    dg_dot.edge(edge[0],edge[1], dir='none')

dg_dot

In [ ]:
# rm,_,_ = pybnl.bn.cpdag(bn)
# print(rm.dimnames[1])
# rm.rx(1,2)
pybnl.bn.cpdag(bn)
# for i in range(12):
#     print(rm.rx(i+1,1), rm.rx(i+1,2))

In [ ]:
pybnl.bn.dag(bn)

In [ ]:
pybnl.bn.score(bn, bn.df)

In [ ]:
pybnl.bn.vstructs(bn)

In [ ]:
tmp1 = bn.rfit
tmp2 = bn.rnet

In [ ]:
%%R -i tmp1 -i tmp2
#print(tmp2)
names(tmp1)

In [ ]:
print(pybnl.bn.rfit2rnet(bn.rfit))

In [ ]:
pybnl.bn.dot(*pybnl.bn.dag(bn))

In [ ]:
pybnl.bn.dot(*pybnl.bn.cpdag(bn))

In [ ]:
print(pybnl.bn.drop_arc(bn.rnet, frm='STAT', to='ANL'))

In [ ]:
pybnl.bn.rnet2cpdag(pybnl.bn.drop_arc(bn.rnet, frm='STAT', to='ANL'))

In [ ]:
pybnl.bn.dot(*pybnl.bn.rnet2cpdag(pybnl.bn.drop_arc(bn.rnet, frm='STAT', to='ANL')),engine='dot')

In [ ]:
tmp1 = bn.rfit
tmp2 = bn.rnet

In [ ]:
%%R -i tmp1 -i tmp2
tmp2_da = drop.arc(tmp2, from="STAT", to="ANL")
# print(tmp2)
cpdag(tmp2_da)